# Análise Exploratória de Dados em Python

Neste notebook, vamos praticar algumas das técnicas e processos de pensamento para análise exploratória de dados. Vamos praticar a análise com um conjunto de dados real. É importante enfatizar que não existe um único caminho correto para a análise exploratória de dados. A maneira como você explora seus dados depende das perguntas que você está tentando responder e dos dados em si. Dessa forma, deixe os dados e sua curiosidade guiá-lo.

Se for necessário acrescentar passos adicionais além daqueles que estão indicados neste notebook, faça isso.


## O Conjunto de Dados

Este conjunto de dados é basicamente uma tabela de amostras com dados de abundância de proteínas nucleares. Essas amostras são de camundongos normais ou de camundongos com uma condição semelhante à _Síndrome de Down_. Alguns desses camundongos receberam uma droga chamada _memantina_, que os pesquisadores conjecturam que poderia resgatar a função cognitiva em camundongos afetados. Existem 38 ratos normais e 34 ratos afetados, e cada rato contribuiu com 15 amostras para o conjunto de dados, para um total de 1080 amostras. A razão de abundância de 77 proteínas é medida para cada amostra.

O conjunto de dados pode ser obtido no [aqui](https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression). Acesse o link `Data Folder`, e salve o arquivo `Data_Cortex_Nuclear.xls` na mesma pasta em que você gravou este notebook. Baixe este arquivo, e carregue-o em um dataframe do Pandas.

In [1]:
import numpy as np
import pandas as pd

df = pd.read_excel('Data_Cortex_Nuclear.xls')

FileNotFoundError: [Errno 2] No such file or directory: 'Data_Cortex_Nuclear.xls'

Em seguida inspecione o dataframe para verificar se há dados faltantes. Caso haja, preencha estes dados com a média dos dados existentes.

In [ ]:
def check_df_nulls(df, verbose=False):
    
    # get columns with float64 values
    is_float = []
    for i in df.dtypes.keys():
        if df.dtypes[i] == 'float64':
            is_float.append(i)

    # get columns with null values      
    has_null = df.isnull().any()
    for i in has_null.keys():
        if has_null[i]:
            print(i, '=>', df[i].isnull().sum(), 'nulls')\
            if verbose else None
        else: has_null.pop(i)

    # get columns with null float64 values
    to_fill = [value for value in is_float if value in has_null.keys()]

    # print overview
    print(len(df.keys()), 'total columns')
    print(len(is_float), 'are float64')
    print(len(has_null), 'contain nulls')
    print(len(to_fill), 'columns to fill\n')
    # return to_fill

# first check
print('Checking data frame...')
check_df_nulls(df)
    
# fill values with mean
print('Filling null values with mean...', end=' ')
df = df.fillna(df.mean())

# for i in to_fill:
#     df[i] = df[i].fillna(df[i].mean())
    
# second check
print('OK.\n\nRechecking...')
check_df_nulls(df)

## Gerando Estatísticas de Resumo Adicionais

Os dataframes do Pandas têm um método prático chamado `describe` que escreve uma variedade de estatísticas de resumo para cada variável. As estatísticas de resumo retornadas por esse método também são um dataframe, portanto, você pode alterá-lo facilmente para incluir outras estatísticas. Vamos fazer exatamente isso, adicionando o coeficiente de variação da variável (cova).

O coeficiente de variação é apenas a razão entre o desvio padrão e a média da distribuição. Distribuições com um grande coeficiente de variação são mais propensas a sinalizar uma mudança no processo de geração de dados subjacente, ao contrário da variação aleatória.

Adicionar uma nova linha no dataframe de `resumo` pode ser realizado da seguinte forma `resumo.loc['cova'] = «dados da nova linha»`.


In [ ]:
resumo = df.describe()
resumo.loc['cova'] = df.std()/df.mean()
resumo

### Normalização

Um passo inicial importante antes de qualquer análise real é realizado para normalizar os dados. Isso é necessário para evitar que variáveis com valores grandes tenham uma influência indevida nos resultados. Existem alguns métodos diferentes de normalizar seus dados:

+ O método mais comum é dividir os valores de uma dimensão pelo maior valor que ela contém. Esse método é fácil de interpretar e é robusto para outliers quando o coeficiente de variação em uma dimensão é baixo.
+ Outro método muito comum é subtrair a média e dividir a dimensão por seu desvio padrão, um processo às vezes chamado de padronização. Esse método fornece mais informações sobre a similaridade relativa de valores quando o coeficiente de variação em uma dimensão é alto. Desempenha mal nas dimensões com um baixo coeficiente de variação e grandes outliers.
+ Uma opção subutilizada é usar a função de distribuição cumulativa da distribuição empírica da dimensão para obter uma classificação de percentil. Esse método funciona bem na maioria dos casos e é robusto para outliers. Sua utilidade se degrada um pouco quando o coeficiente de variação de uma dimensão fica muito pequeno.

O método de normalização usado depende dos dados e dos aspectos que você deseja destacar. Você pode usar um método de normalização diferente para cada dimensão de seus dados, embora eu não recomende isso, a menos que seja absolutamente necessário. Em geral, minha preferência é pelo método de distribuição empírica.

O código em Python necessário para fazer os 3 tipos de normalização é exemplificado abaixo (supondo que o dataframe original se chame `df` e contenha apenas dados numéricos):

    minmax_normalized_df = pd.DataFrame(MinMaxScaler().fit_transform(df), 
    columns=df_numeric.columns, index=df_numeric.index)
    
    standardized_df = pd.DataFrame(StandardScaler().fit_transform(df), 
    columns=df_numeric.columns, index=df_numeric.index)
    
    ecdf_normalized_df = df.apply(lambda c: pd.Series(ECDF(c)(c), index=c.index))


In [ ]:
# get columns with float values from data frame
df_floats = df.loc[:, df.dtypes == float]

# normalize values in data frame (MinMax)
from sklearn.preprocessing import MinMaxScaler
normalized_df = pd.DataFrame(MinMaxScaler().fit_transform(df_floats),
                             columns=df_floats.columns, index=df.index)

# normalize values in data frame (Standard)
# from sklearn.preprocessing import StandardScaler
# normalized_df = pd.DataFrame(StandardScaler().fit_transform(df_floats),
#                              columns=df_floats.columns, index=df.index)

# normalize data frame (ECDF)
# from statsmodels.distributions.empirical_distribution import ECDF
# normalized_df = df_floats.apply(lambda c: pd.Series(ECDF(c)(c), index=c.index))

### Explorando dados

Usaremos a estratégia de explorar dados procurando coisas que são fora do comum. Em um conjunto de dados que seja apenas variáveis aleatórias gaussianas não correlacionadas, não poderiamos aprender nada com ele, seria literalmente ruído branco. A maior parte do conteúdo da informação na maioria dos conjuntos de dados é sua variação da normalidade.

A distribuição do coeficiente de variação informa sobre a variabilidade relativa desses dados. Altos coeficientes de variação para uma variável indicam um provável candidato a efeito experimental.

Plote o histograma do coeficiente de variação dos dados.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

# get coefficient_of_variation
series_cova = normalized_df.std()/normalized_df.mean()
series_cova.head(1)

In [ ]:
# plot histogram
plt.hist(series_cova, bins=30, alpha=0.7, rwidth=0.9)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Coeficiente de variação')
plt.ylabel('Frequência')
plt.title('Histograma')
# plt.xlim(left=0.15, right=0.55) # <-- slice it further
plt.show()

# plot more histograms
# for i in df.keys():
#     if df.dtypes[i] == 'float64':
#         plt.hist(normalized_df[i], bins='auto', alpha=0.7, rwidth=0.9)
#         plt.grid(axis='y', alpha=0.75)
#         plt.xlabel(str(i))
#         plt.ylabel('Frequency')
#         plt.title(i)
#         plt.show()

Do gráfico acima é possível estimar um valor de corte para o coeficiente de variação? Um valor abaixo do qual poderíamos desprezar a alteração das proteínas como sendo um efeito do tratamento? Qual seria este valor?

Se você achar que sim, gere um novo dataframe apenas com as proteínas que interessam segundo este critério (A).


**Resposta**: Considerando o gráfico acima, argumenta-se que o valor de corte deve visar as proteínas que obtiveram um coeficiente de variação de aproximadamente >= 0.496 entre si, por razão:
- a) da grande quantidade de proteínas que exibiram esse padrão (de pouco variar entre si);
- b) de que distribuições com um grande coeficiente de variação são mais propensas a sinalizar uma mudança no processo de geração de dados subjacente, ao contrário da variação aleatória.

Nota-se que uma análise exploratória de cada variável em mais histogramas no conjunto subsequente pode ser útil para o entendimento dos dados.

In [ ]:
df_A = df.copy()
proteins = []

# considering <= 0.4
float_cova = 0.4
for x in series_cova.items():
    if x[0] in df_A.keys()\
    and x[1] <= float_cova:
        df_A.pop(x[0])
        proteins.append(x[0])

int_total_old = len(df_floats.keys())
int_discarded = len(proteins)
int_total_new = int_total_old - int_discarded

print(int_total_old, 'total proteins')
print(int_discarded, 'proteins cova <=', float_cova, '(DISCARDED)')
print(int_total_new, 'proteins cova >=', float_cova, '(OK)')

Outro passo recomendável é realizar testes de normalidade nas variáveis. Variáveis que se desviam muito da normalidade são mais prováveis de serem informativas. O teste de Anderson-Darling ou o teste de Shapiro-Wilks funciona bem para esse propósito.

Realize ambos os testes abaixo, plote um histograma com os resultados dos testes para as 77 variáveis (proteínas), e verifique:
1. Se os dois testes diferem significativamente em seus resultados.
2. Se os testes permitem estimar um valor de corte abaixo do qual podemos desprezar as variáveis como “provavelmente não-informativas”.


In [ ]:
# # anderson-darling
# The Anderson-Darling tests the null hypothesis that a sample is drawn from a population
# that follows a particular distribution. If the returned statistic is larger than these
# critical values then for the corresponding significance level, the null hypothesis that
# the data come from the chosen distribution can be rejected.

from scipy.stats import anderson
from collections import defaultdict

# test if possible to reject H0
dict_anderson = defaultdict(dict)
for x in df.loc[:, df.dtypes == float].keys().tolist():
    anderson_stats = anderson(df[x])
    anderson_statistic = anderson_stats.statistic
    dict_anderson['statistic'][x] = anderson_statistic
#     print('\n'+x, 'Statistic: %.3f' % anderson_statistic)
#     for i in range(len(anderson_stats.critical_values)):
#         cv = anderson_stats.critical_values[i]
#         sl = anderson_stats.significance_level[i]
#         if anderson_statistic < cv:
#             print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
#         else: print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))

# save lists of critical values and significance levels
anderson_cv = anderson_stats.critical_values.tolist()
anderson_sl = anderson_stats.significance_level.tolist()
        
# transform to series and append to described df
series_anderson = pd.Series(dict_anderson['statistic'])
resumo.loc['anderson'] = series_anderson

In [ ]:
# shapiro-wilks
# Esta função retorna dois valores, o primeiro é a Estatística F da amostra
# e o segundo é o valor 𝑝 da hipótese de que a amostra seja normalmente distribuída.
# Assim, se o segundo valor retornado pela função shapiro for superior ao valor de alfa
# isso indica que a amostra deve ser, de fato, normalmente distribuída.

from scipy.stats import shapiro

# get p value for hypothesis
dict_shapiro = defaultdict(dict)
for x in df.loc[:, df.dtypes == float].keys().tolist():
    f, p = shapiro(df[x])
    dict_shapiro['f'][x] = f
    dict_shapiro['p'][x] = p

# transform to series and append to described df
series_shapiro = pd.Series(dict_shapiro['p'])
resumo.loc['shapiro_p'] = series_shapiro
series_shapiro.head(1)


In [ ]:
# shapiro histogram
plt.hist(series_shapiro, bins='auto', alpha=0.7, rwidth=0.9) # <-- plotar aqui pareceu levar uma eternidade!
plt.grid(axis='y', alpha=0.75)
plt.xlabel('shapiro p value')
plt.ylabel('Frequência')
plt.title('Histograma')
plt.show()

In [ ]:
# anderson histogram
plt.hist(series_anderson, bins='auto', alpha=0.7, rwidth=0.9) # <-- plotar aqui pareceu levar uma eternidade!
plt.grid(axis='y', alpha=0.75)
plt.xlabel('anderson statistic')
plt.ylabel('Frequência')
plt.title('Histograma')
plt.show()

Se for possível estabelecer um valor de corte para as variáveis, gere um novo dataframe apenas com as proteínas que interessam segundo este critério (B).

In [ ]:
df_B = df.copy()
proteins = []

# considering shapiro p > 0.05
for x in series_shapiro.items():
    if x[1] > 0.05:
        df_B.pop(x[0])
        proteins.append(x[0])

int_total_old = len(df_floats.keys())
int_discarded = len(proteins)
int_total_new = int_total_old - int_discarded

print(int_total_old, 'total proteins')
print(int_discarded, 'proteins look normal (fail to reject H0)')
print(int_total_new, 'proteins do not look normal (reject H0)')

Existe sobreposição entre variáveis com alto coeficiente de variação e variáveis altamente não-normais? Caso exista, qual é esta sobreposição?

Se criarmos conjuntos a partir dos dataframes gerados pela aplicação dos critério (A) e (B) acima, poderemos ver quais proteínas estão presentes em ambos os conjuntos. Faça isso.

In [ ]:
df_C = df.copy()

proteins = df_floats.keys().tolist()
proteins_A = df_A.keys().tolist()
proteins_B = df_B.keys().tolist()

for x in proteins:
    if not all(x in k for k in [proteins_A, proteins_B]):
        df_C.pop(x)

proteins_C = df_C.loc[:, df.dtypes == float].keys().tolist()
print('Proteins =>', proteins_C)

Provavelmente, você chegou a um número de proteínas bem menor do que 77, correto? Essas são as proteínas que tem melhores chances de indicar resultados no tratamento. O que fizemos aqui se chama _redução de dimensionalidade_.

Isso nos deixa com um conjunto menor de variáveis que foram selecionadas através de múltiplos mecanismos. Vamos em frente e visualizá-los todos ao mesmo tempo. Gere, em um único gráfico, o histograma para as concentrações de todas as proteínas candidatas que foram selecionadas até aqui.

In [ ]:
%matplotlib inline

# plot histogram
plt.hist(df_C.mean(), bins='auto', alpha=0.7, rwidth=0.9)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Concentração média')
plt.ylabel('Frequência')
plt.title('Histograma')
plt.show()

Outras métricas interessantes a serem consideradas incluem o _skew_ (inclinação) e _kurtosis_ (curtose) das distribuições. A inclinação mede a simetria de uma distribuição sobre sua média, enquanto a curtose mede a parte dos dados nas caudas da distribuição.

Skew e kurtosis podem ser combinadas em um bom gráfico de dispersão que informa muito sobre a estrutura do seu conjunto de dados.

In [ ]:
### Defina abaixo o dataframe contendo apenas os atributos numéricos (concentração de proteínas)
### Sinta-se à vontade para melhorar o gráfico se quiser.

df_numeric = df_floats # df.loc[:, df.dtypes == float]

skews = df_numeric.apply(lambda x: pd.DataFrame.skew(x))
skews.name = "skew"

kurts = df_numeric.apply(lambda x: pd.DataFrame.kurtosis(x))
kurts.name = "kurtosis"

proteins = pd.Series([i.split("_")[0] for i in kurts.index], index=kurts.index, name="protein")
sk_df = pd.concat([skews, kurts, proteins], axis=1)
plt.scatter(sk_df['skew'], sk_df['kurtosis'])

No _scatter plot_ acima é possível identificar com certa facilidade os outliers.

Três outliers identificados fora da curva exibida:
1. x=~2.5, y=~35
2. x=~2.7, y=~43
3. x=~4.8, y=~62

In [ ]:
# write proteins to output file
with open('candidate-columns.csv', 'w') as f:
    f.write('\n'.join('%s' % protein for protein in proteins_C))